In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.0 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").getOrCreate()

In [8]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [9]:
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R3EQSTM9PWRAL|          3|            0|          0|   N|                Y|
| RBWPRK17XKIXD|          5|            0|          0|   N|                Y|
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|                Y|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|                Y|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|                Y|
|R1L94ESVC657A9|          5|            0|          0|   N|                Y|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|                Y|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|                Y|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|     

In [10]:
vine_twenty_votes= vine_df.filter(df.total_votes >= 20)
vine_twenty_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
|R1M8EEGH2TVS0Q|          1|            6|         29|   N|                Y|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|     

In [18]:
vine_helpful = vine_twenty_votes.filter((vine_twenty_votes.helpful_votes / vine_twenty_votes.total_votes) >= 0.5)

vine_helpful.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|                Y|
| RYA9ESIATW4EC|          1|           23|         27|   N|     

In [23]:
vine_yes =vine_helpful.filter("vine == 'Y'")

vine_yes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RG70K1HMY4LXX|          4|           37|         44|   Y|                N|
|R1DHGTNXDXJ0GB|          5|           78|         93|   Y|                N|
|R3B5S3OSA8N6AO|          5|           38|         43|   Y|                N|
|R29X0YBS0SBIUP|          5|           56|         74|   Y|                N|
|R1ZEDMCU6NYC8Q|          3|           26|         27|   Y|                N|
| RVBJM1LUI9X9K|          5|           27|         28|   Y|                N|
| R1KASHD9QSDMV|          4|           18|         21|   Y|                N|
|R1M3LATUI38MK0|          5|           46|         50|   Y|                N|
|R20JZQ3AQVYNAU|          4|          103|        113|   Y|                N|
|R1EHXYTMBO4D28|          1|           56|         61|   Y|     

In [27]:
vine_no =vine_helpful.filter("vine == 'N'")
vine_no.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|                Y|
| RYA9ESIATW4EC|          1|           23|         27|   N|     

In [34]:
#Total Vote Count 
 vine_df.count()


5331449

In [56]:
# Number of Five Star Reviews
five_star = vine_df.filter('star_rating == 5').count()

five_star



3358086

In [57]:
# Percentage of Vine Five Star Reviews
vine_df.filter((vine_df['star_rating'] == 5) & (vine_df['vine']=='Y')).count() / five_star * 100

0.3909072013045527

In [58]:
# Percentage of Non Vine Five Star Reviews 
vine_df.filter((vine_df['star_rating'] == 5) & (vine_df['vine']=='N')).count() / five_star *100

99.60909279869544